In [22]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from flask import Flask, jsonify, render_template
import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd

In [8]:
connection_string = "postgres:postgres@localhost:5432/ETL_Rental_DB"

In [14]:
def postgres_create_session(connection_string):
    #####This functions create all the background functions for a successful connections to the db
    #####and returns a session class, mapped classes
    #Create an engine to the hawaii.sqlite database
    engine = create_engine(f'postgresql://{connection_string}', echo=True)
    # reflect an existing database into a new model; reflect the tables
    Base = automap_base()
    Base.prepare(engine, reflect=True)

    # Save references to each table
    Rental = Base.classes.Rental
    Income = Base.classes.Income
    Crime = Base.classes.Crime
    Community_Assets = Base.classes.Community_Assets
    Bridge_Rental_Crime = Base.classes.Bridge_Rental_Crime

    # Create our session (link) from Python to the DB
    session = Session(bind=engine)
    return session, Rental, Income, Crime, Community_Assets, Bridge_Rental_Crime

In [15]:
session, Rental, Income, Crime, Community_Assets, Bridge_Rental_Crime = postgres_create_session(connection_string)

2020-08-15 20:44:56,239 INFO sqlalchemy.engine.base.Engine select version()
2020-08-15 20:44:56,240 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,243 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-08-15 20:44:56,245 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,255 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-15 20:44:56,256 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,262 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-15 20:44:56,264 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,273 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-08-15 20:44:56,275 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,284 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2020-0

2020-08-15 20:44:56,453 INFO sqlalchemy.engine.base.Engine {'table_name': 'Rental'}
2020-08-15 20:44:56,462 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
              pgd.description as comment,
              a.attgenerated as generated
            FROM pg_catalog.pg_attribute a
            LEFT JOIN pg_catalog.pg_description pgd ON (
                pgd.objoid = a.attrelid AND pgd.objsubid = a.attnum)
            WHERE a.attrelid = %(table_oid)s
            AND a.attnum > 0 AND NOT a.attisdropped
            ORDER BY a.attnum
        
2020-08-15 20:44:56,466 INFO sqlalchemy.engine.base.Engine {'table_oid': 274

2020-08-15 20:44:56,643 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,658 INFO sqlalchemy.engine.base.Engine 
            SELECT t.typname as "name",
               -- no enum defaults in 8.4 at least
               -- t.typdefault as "default",
               pg_catalog.pg_type_is_visible(t.oid) as "visible",
               n.nspname as "schema",
               e.enumlabel as "label"
            FROM pg_catalog.pg_type t
                 LEFT JOIN pg_catalog.pg_namespace n ON n.oid = t.typnamespace
                 LEFT JOIN pg_catalog.pg_enum e ON t.oid = e.enumtypid
            WHERE t.typtype = 'e'
        ORDER BY "schema", "name", e.oid
2020-08-15 20:44:56,661 INFO sqlalchemy.engine.base.Engine {}
2020-08-15 20:44:56,672 INFO sqlalchemy.engine.base.Engine 
                SELECT a.attname
                FROM pg_attribute a JOIN (
                    SELECT unnest(ix.indkey) attnum,
                           generate_subscripts(ix.indkey, 1) ord
                    F

2020-08-15 20:44:56,798 INFO sqlalchemy.engine.base.Engine 
        SELECT conname
           FROM  pg_catalog.pg_constraint r
           WHERE r.conrelid = %(table_oid)s AND r.contype = 'p'
           ORDER BY 1
        
2020-08-15 20:44:56,800 INFO sqlalchemy.engine.base.Engine {'table_oid': 27420}
2020-08-15 20:44:56,805 INFO sqlalchemy.engine.base.Engine 
          SELECT r.conname,
                pg_catalog.pg_get_constraintdef(r.oid, true) as condef,
                n.nspname as conschema
          FROM  pg_catalog.pg_constraint r,
                pg_namespace n,
                pg_class c

          WHERE r.conrelid = %(table)s AND
                r.contype = 'f' AND
                c.oid = confrelid AND
                n.oid = c.relnamespace
          ORDER BY 1
        
2020-08-15 20:44:56,808 INFO sqlalchemy.engine.base.Engine {'table': 27420}
2020-08-15 20:44:56,813 INFO sqlalchemy.engine.base.Engine 
              SELECT
                  i.relname as relname,
            

2020-08-15 20:44:56,934 INFO sqlalchemy.engine.base.Engine {'table_oid': 27428}
2020-08-15 20:44:56,940 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        
2020-08-15 20:44:56,943 INFO sqlalchemy.engine.base.Engine {'table_oid': 27428}
2020-08-15 20:44:56,946 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND

In [37]:
def listings(Rental, count=100):
    ####This function retrieves the "count" of listings upto 100 of data from the Rental class
  
    
    #limit count to 100
    count = 100 if count>100 else max(count,1)
    ### Design a query to retrieve the "count" no of listings
    rental_listing = session.query(Rental.id, Rental.title, Rental.price, Rental.image, Rental.url, Rental.bedrooms, Rental.rental_type, Rental.source, Rental.sqft).filter().order_by(Rental.post_published_date).limit(count)
    rental_listing_DF = pd.DataFrame(rental_listing)

    #Convert the DF to a dictionary
    rental_listing_dict = rental_listing_DF.T.to_dict()
    return rental_listing_dict

In [58]:
def comm_services(Community_Assets, count=100):
    ####This function retrieves the "count" of comm_services upto 100 of data from the Community_Assets class
  
    
    #limit count to 100
    count = 100 if count>100 else max(count,1)
    ### Design a query to retrieve the "count" no of services
    service_listing = session.query(Community_Assets.id, Community_Assets.agency_name, Community_Assets.e_mail, Community_Assets.fees, Community_Assets.hours, Community_Assets.application,  Community_Assets.category, Community_Assets.address, Community_Assets.crisis_phone).limit(count)
    servicel_listing_DF = pd.DataFrame(service_listing)

    #Convert the DF to a dictionary
    servicel_listing_dict = servicel_listing_DF.T.to_dict()
    return servicel_listing_dict

In [65]:
def crime_details(Crime, Type="Assault"):
    ####This function retrieves all the crime data in the last year based on type
    #["assault", "auto theft", "break and enter", "robbery" ,'Homicide', and "theft over"]
  
    Type = "Assault" if Type not in ['Assault', 'Auto Theft', 'Break and Enter', 'Theft Over',
       'Robbery', 'Homicide'] else Type
    

    ### Design a query to retrieve all the crime data based on type
    crime_listing = session.query(Crime.MCI, Crime.occurrencedate, Crime.reporteddate, Crime.offence, Crime.neighbourhood).filter(Crime.MCI==Type).order_by(Crime.occurrencedate)
    crime_listing_DF = pd.DataFrame(crime_listing)

    #Convert the DF to a dictionary
    crime_listing_dict = crime_listing_DF.T.to_dict()
    return crime_listing_dict

In [70]:
def income_details(Income):
    ####This function retrieves the income details for all FSA in Toronto
    

    ### Design a query to retrieve all the income data for all FSA in Toronto
    fsa_income = session.query(Income.FSA, Income.avg_income)
    fsa_income_DF = pd.DataFrame(fsa_income)

    #Convert the DF to a dictionary
    fsa_income_dict = fsa_income_DF.T.to_dict()
    return fsa_income_dict

In [38]:
listings(Rental, count=100)

2020-08-15 20:58:18,035 INFO sqlalchemy.engine.base.Engine SELECT "Rental".id AS "Rental_id", "Rental".title AS "Rental_title", "Rental".price AS "Rental_price", "Rental".image AS "Rental_image", "Rental".url AS "Rental_url", "Rental".bedrooms AS "Rental_bedrooms", "Rental".rental_type AS "Rental_rental_type", "Rental".source AS "Rental_source", "Rental".sqft AS "Rental_sqft" 
FROM "Rental" ORDER BY "Rental".post_published_date 
 LIMIT %(param_1)s
2020-08-15 20:58:18,037 INFO sqlalchemy.engine.base.Engine {'param_1': 100}


{0: {'id': 'k_1502301137',
  'title': 'One, two bedroom large unit from $1650 , Hydro extra fully reno',
  'price': 1650,
  'image': 'https://i.ebayimg.com/00/s/NjAwWDgwMA==/z/cIkAAOSw~HheyA5k/$_27.JPG',
  'url': 'https://www.kijiji.ca/v-apartments-condos/city-of-toronto/one-two-bedroom-large-unit-from-1650-hydro-extra-fully-reno/1502301137',
  'bedrooms': 1.0,
  'rental_type': 'apartment',
  'source': 'Kijiji',
  'sqft': 750.0},
 1: {'id': 'k_1512876086',
  'title': 'Studio at 111 Elizabeth St. Toronto',
  'price': 1750,
  'image': 'https://i.ebayimg.com/00/s/MzAyWDQ1NA==/z/-wkAAOSwvAtfExFG/$_27.JPG',
  'url': 'https://www.kijiji.ca/v-apartments-condos/city-of-toronto/studio-at-111-elizabeth-st-toronto/1512876086',
  'bedrooms': 0.0,
  'rental_type': 'condo',
  'source': 'Kijiji',
  'sqft': 400.0},
 2: {'id': 'k_1513238004',
  'title': 'Newly built Studio Apartment - Yonge and Wellesley',
  'price': 1600,
  'image': 'https://i.ebayimg.com/00/s/MTYwMFg3NDA=/z/FTYAAOSwChpfFfQj/$_27.JPG'

In [59]:
comm_services(Community_Assets, count=100)

2020-08-15 21:13:06,686 INFO sqlalchemy.engine.base.Engine SELECT "Community_Assets".id AS "Community_Assets_id", "Community_Assets".agency_name AS "Community_Assets_agency_name", "Community_Assets".e_mail AS "Community_Assets_e_mail", "Community_Assets".fees AS "Community_Assets_fees", "Community_Assets".hours AS "Community_Assets_hours", "Community_Assets".application AS "Community_Assets_application", "Community_Assets".category AS "Community_Assets_category", "Community_Assets".address AS "Community_Assets_address", "Community_Assets".crisis_phone AS "Community_Assets_crisis_phone" 
FROM "Community_Assets" 
 LIMIT %(param_1)s
2020-08-15 21:13:06,688 INFO sqlalchemy.engine.base.Engine {'param_1': 100}


{0: {'id': '0',
  'agency_name': 'Child and Family Services Review Board',
  'e_mail': 'cfsrb@ontario.ca',
  'fees': 'Unknown',
  'hours': 'Mon-Fri 9 am-5 pm',
  'application': 'Call, email, or visit website for information * applications available online to download -- submit completed application in person, or by fax, email, mail or courier',
  'category': 'Community Services',
  'address': '655 Bay St, 14th Fl Toronto, ON M7A 2A3',
  'crisis_phone': None},
 1: {'id': '1',
  'agency_name': 'Custody Review Board',
  'e_mail': 'cfsrb@ontario.ca',
  'fees': 'Unknown',
  'hours': 'Mon-Fri 9 am-5 pm',
  'application': 'Call or visit website for information, including information required to complete a telephone application through the Case Coordinator * collect calls accepted',
  'category': 'Community Services',
  'address': '655 Bay St, 14th Fl Toronto, ON M7A 2A3',
  'crisis_phone': None},
 2: {'id': '2',
  'agency_name': 'Social Justice Tribunals Ontario',
  'e_mail': 'sjtocomplaints@

In [66]:
crime_details(Crime, "Homicide")

2020-08-15 21:24:40,833 INFO sqlalchemy.engine.base.Engine SELECT "Crime"."MCI" AS "Crime_MCI", "Crime".occurrencedate AS "Crime_occurrencedate", "Crime".reporteddate AS "Crime_reporteddate", "Crime".offence AS "Crime_offence", "Crime".neighbourhood AS "Crime_neighbourhood" 
FROM "Crime" 
WHERE "Crime"."MCI" = %(MCI_1)s ORDER BY "Crime".occurrencedate
2020-08-15 21:24:40,837 INFO sqlalchemy.engine.base.Engine {'MCI_1': 'Homicide'}


{0: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 1, 6),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Kennedy Park (124)'},
 1: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 1, 8),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Clairlea-Birchmount (120)'},
 2: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 1, 9),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Rexdale-Kipling (4)'},
 3: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 1, 12),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Dorset Park (126)'},
 4: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 1, 24),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Rockcliffe-Smythe (111)'},
 5: {'MCI': 'Homicide',
  'occurrencedate': datetime.date(2019, 2, 8),
  'reporteddate': None,
  'offence': 'Homicide',
  'neighbourhood': 'Regent Park (72)'},
 6: {'MCI': 'Hom

In [32]:
#limit count to 100
count = 100
count = 100 if count>100 else max(count,1)
rental_listing = session.query(Rental.id, Rental.title, Rental.price, Rental.image, Rental.url, Rental.bedrooms, Rental.rental_type, Rental.source, Rental.sqft).filter().order_by(Rental.post_published_date).limit(count)
rental_listing_DF = pd.DataFrame(rental_listing)


2020-08-15 20:54:02,989 INFO sqlalchemy.engine.base.Engine SELECT "Rental".id AS "Rental_id", "Rental".title AS "Rental_title", "Rental".price AS "Rental_price", "Rental".image AS "Rental_image", "Rental".url AS "Rental_url", "Rental".bedrooms AS "Rental_bedrooms", "Rental".rental_type AS "Rental_rental_type", "Rental".source AS "Rental_source", "Rental".sqft AS "Rental_sqft" 
FROM "Rental" ORDER BY "Rental".post_published_date 
 LIMIT %(param_1)s
2020-08-15 20:54:02,990 INFO sqlalchemy.engine.base.Engine {'param_1': 100}


In [72]:
income_details(Income)

2020-08-15 22:02:13,573 INFO sqlalchemy.engine.base.Engine SELECT "Income"."FSA" AS "Income_FSA", "Income".avg_income AS "Income_avg_income" 
FROM "Income"
2020-08-15 22:02:13,576 INFO sqlalchemy.engine.base.Engine {}


{0: {'FSA': 'H4E', 'avg_income': 32297.449610000003},
 1: {'FSA': 'J3Y', 'avg_income': 41920.621869999995},
 2: {'FSA': 'K0M', 'avg_income': 41941.82174},
 3: {'FSA': 'L0A', 'avg_income': 50645.79946},
 4: {'FSA': 'L0G', 'avg_income': 58406.25992999999},
 5: {'FSA': 'L0H', 'avg_income': 72340.64748},
 6: {'FSA': 'L0J', 'avg_income': 115162.7249},
 7: {'FSA': 'L0N', 'avg_income': 56237.54513},
 8: {'FSA': 'L0P', 'avg_income': 73380.24132999999},
 9: {'FSA': 'L1G', 'avg_income': 42261.794630000004},
 10: {'FSA': 'L1H', 'avg_income': 39385.14449},
 11: {'FSA': 'L1J', 'avg_income': 39423.65492},
 12: {'FSA': 'L1K', 'avg_income': 49227.0194},
 13: {'FSA': 'L1L', 'avg_income': 51335.44304},
 14: {'FSA': 'L1N', 'avg_income': 50608.74933},
 15: {'FSA': 'L1R', 'avg_income': 56518.79973},
 16: {'FSA': 'L1S', 'avg_income': 46207.4259},
 17: {'FSA': 'L1T', 'avg_income': 46148.88948},
 18: {'FSA': 'L1V', 'avg_income': 52087.33154},
 19: {'FSA': 'L1W', 'avg_income': 52819.409589999996},
 20: {'FSA':